# Картографирование потенциала плотности расселения

**Date**: 19.04.2023

**Course**: Methods of Spatial Analysis. Advanced Level. // HSE, Moscow, spring 2023

In [ ]:
# Restart the kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)
print('Ядро перезапущено, можно продолжать работать!')

## 1. Import libraries

In [2]:
import pandas as pd
import geopandas as gpd
import folium
import math

## 2. Read and filter data

In [3]:
data = pd.read_csv('cities.csv', sep=';') 
# selectedData = data.loc[(data['region'] == 'Краснодарский край') & (data['type'] == 'г')] 
selectedData = data.loc[(data['region'] == 'Краснодарский край')] 

## 3. Explore the data

In [ ]:
selectedData.head()

## 4. Create spatial data frame (and reproject)

In [6]:
spatialData = gpd.GeoDataFrame(selectedData, geometry=gpd.points_from_xy(selectedData.longitude_dd, selectedData.latitude_dd), crs='EPSG:4326').to_crs(crs='EPSG:32637')

## 5. Explore the data

In [ ]:
spatialData.head()

In [8]:
spatialData.explore()

## 6. Data processing

In [9]:
spatialData_new = spatialData.set_index('id')

#### 6.1 Distance matrix

In [ ]:
distanceMatrix = spatialData_new.geometry.apply(lambda g: spatialData_new.distance(g))
distanceMatrix

#### 6.2 Join population to distance matrix

In [11]:
merged = pd.merge(distanceMatrix, spatialData_new[['population']], left_index=True, right_index=True)

In [ ]:
merged.head()

#### 6.3 Function to calculate population density potential (for each row)

In [13]:
def calculate_potential(row):
    array = merged.columns
    sum = 0
    for val in array:
        distance = row[val]
        if distance == 0:
            continue
        pop = merged.loc[merged[val] == row[val], 'population'].iloc[0]
        sum += pop/distance
    
    return math.log2(sum*row['population'])

#### 6.4 Create a subset to test algorithm

In [14]:
subset = merged.iloc[:10]

#### 6.5 Testing algorithm 

In [ ]:
subset['potential'] = subset.apply(calculate_potential, axis=1)

#### 6.6 Check the result

In [ ]:
subset.head()

#### 6.7 Apply algorithm to a main dataset

In [17]:
merged['potential'] = merged.apply(calculate_potential, axis=1)

In [ ]:
merged.head()

## 7 Join the result to initial data 

In [20]:
spatialPotential = spatialData_new.merge(merged['potential'], left_index=True, right_index=True)

## 8 Save dataset to a file

In [22]:
spatialPotential.to_file('pop_density_potential.gpkg')